# Kinetic temperature

The kinetic temperature is obtained with the 12CO amplitude :
$$T_{kin}=T_{ex}=\frac{5.53}{\ln\left(1+\frac{5.53}{T_R+0.148}\right)}$$

In [ ]:
from src.hdu.maps.map import Map
from src.hdu.cubes.cube_co import CubeCO
from src.hdu.tesseract import Tesseract
from src.hdu.maps.grouped_maps import GroupedMaps
from src.hdu.maps.convenient_funcs import get_kinetic_temperature

In [ ]:
GroupedMaps([(
    "kinetic_temperature", [
        get_kinetic_temperature(amp) for amp in Tesseract.load(
            "data/Loop4_co/N1/12co/object_filtered.fits"
        ).to_grouped_maps().amplitude
    ]
)]).save("t.fits")

# Column density

The column density is obtained with
\begin{align*}
    N(^{13}\mathrm{CO})=\frac{3h\Delta V_{13}}{8\pi^3\mu^2}\left(\frac{Qh\nu_{13}/k}{1-e^{-\frac{h\nu_{13}}{kT_{ex}}}}\right)
\end{align*}
Knowing that
$$Q=\frac{2T_{ex}^{13}}{h\nu_{13}/k}$$
we can simplify to
\begin{align*}
    N(^{13}\mathrm{CO})&=\frac{3h\Delta V_{13}}{8\pi^3\mu^2}\left(\frac{\frac{2T_{ex}^{13}}{h\nu_{13}/k}h\nu_{13}/k}{1-e^{-\frac{h\nu_{13}}{kT_{ex}}}}\right)\\
    &=\frac{3h\Delta V_{13}T_{ex}^{13}}{4\pi^3\mu^2\left(1-e^{-\frac{h\nu_{13}}{kT_{ex}}}\right)}\\
\end{align*}
Unit treatment gives:
\begin{align*}
    \left[N(^{13}\mathrm{CO})\right]&=\mathrm{\frac{J\cdot s\cdot km\cdot s^{-1}\cdot K}{(10^{-18}\cdot cm^{5/2}\cdot g^{1/2}\cdot s^{-1})^2}}\\
    &=\mathrm{\frac{(kg\cdot m^2\cdot s^{-2})\cdot km\cdot K}{10^{-36}\cdot cm^{5}\cdot g\cdot s^{-2}}}\\
    &=\mathrm{\frac{(10^3g\cdot 10^4cm^2)\cdot 10^5cm\cdot K}{10^{-36}\cdot cm^{5}\cdot g}}\\
    &=\mathrm{\frac{10^{48}\cdot K}{cm^{2}}}\\
\end{align*}

In [ ]:
import numpy as np
import graphinglib as gl
import importlib

import src.hdu.maps.convenient_funcs
importlib.reload(src.hdu.maps.convenient_funcs)

In [ ]:
cube_12co = CubeCO.load("data/Loop4_co/N1/12co/Loop4N1_wcs.fits")
cube_13co = CubeCO.load("data/Loop4_co/N1/13co/Loop4N1_13co_corrected.fits")
maps_12co = Tesseract.load("data/Loop4_co/N1/12co/object_filtered.fits").to_grouped_maps()
maps_13co = Tesseract.load("data/Loop4_co/N1/13co/tesseract_corrected.fits").to_grouped_maps()

# The right gaussians first need to be selected
# This solution is for single component 13co maps
assert len(maps_13co.mean) == 1
mean_12co = np.stack([m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.mean], axis=0)
diff_array= np.abs(mean_12co - maps_13co.mean[0].data)  # Compute the difference between the centroid of every gaussian
nan_mask = np.isnan(diff_array)     # Apply a nan mask to allow proper argmin use
diff_array[nan_mask] = 2**15-1      # Remove nans
min_mask = np.argmin(diff_array, axis=0)
filter_gaussians = lambda arr: np.take_along_axis(arr, min_mask[np.newaxis, ...], axis=0).squeeze()
amp_12co_val = np.stack([m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.amplitude], axis=0)
amp_12co_unc = np.stack([m.get_reprojection_on(maps_13co.mean[0]).uncertainties for m in maps_12co.amplitude], axis=0)
# print(amp_12co_val[0,19,27])
# print(maps_13co.stddev[0][19,27])

t = src.hdu.maps.convenient_funcs.get_13co_column_density(
    fwhm_13co=maps_13co.stddev[0]*2*np.sqrt(2*np.log(2))*np.abs(cube_12co.header["CDELT3"]/1000),
    kinetic_temperature_13co=get_kinetic_temperature(maps_13co.amplitude[0]/0.43),
    kinetic_temperature_12co=get_kinetic_temperature(Map(filter_gaussians(amp_12co_val), filter_gaussians(amp_12co_unc)))
).save("data/Loop4_co/N1/13co/N1_column_density.fits")

### Global check

In [ ]:
from src.coordinates.ds9_coords import DS9Coords

coords = DS9Coords(28,20)
h_coords = DS9Coords(14,10)

spec_1 = CubeCO.load("data/Loop4_co/N1/12co/Loop4N1_wcs.fits")[500:800,*coords]
spec_2 = CubeCO.load("data/Loop4_co/N1/13co/Loop4N1_13co_corrected.fits")[:,*coords]

fig = gl.Figure()
# fig.add_elements(*Tesseract.load("data/Loop4_co/N1/12co/object_filtered.fits").get_spectrum_plot(cube_12co.bin((1,2,2))[500:800,:,:], h_coords))
fig.add_elements(*Tesseract.load("data/Loop4_co/N1/13co/tesseract.fits").get_spectrum_plot(cube_13co, coords))
# %matplotlib tk
# fig.show()

# fig = gl.Figure()
# fig.add_elements(spec_1.plot)
# fig.show()
# fig = gl.Figure()
# fig.add_elements(spec_2.plot)
# fig.show()
a_12co = 6.43
a_13co = 1.03
s_13co = 8.5